In [16]:
!pip install keyboard


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
   ---------------------------------------- 58.1/58.1 kB 1.5 MB/s eta 0:00:00


In [1]:
import cv2
import face_recognition
import numpy as np
import os
import threading
import keyboard
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
import cv2
import face_recognition
import numpy as np
import os
import threading
from tkinter import Tk, Label, Entry, Button, filedialog, Toplevel
import keyboard

# Flag to indicate if 'q' has been pressed
stop_flag = False

# Function to load known faces
def load_known_faces(folder):
    known_encodings = []
    known_names = []
    aadhaar_info = {}
    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.png')):
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:
                known_encodings.append(encoding[0])
                name = os.path.splitext(filename)[0]  # Use filename without extension as the name
                known_names.append(name)
                aadhaar_info[name] = f"Aadhaar Details for {name}: Dummy Aadhaar {name}"
    return known_encodings, known_names, aadhaar_info

# Function to match faces
def match_faces(image_path, threshold, known_encodings, known_names):
    image = face_recognition.load_image_file(image_path)
    image_encodings = face_recognition.face_encodings(image)

    for face_encoding in image_encodings:
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=threshold)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_names[best_match_index]
            return name, best_match_index
    return "Unknown", None

# Function to process a selected image and match it
def process_selected_image(image_path, threshold):
    recognized_faces.clear()
    results = []
    matched_name, matched_index = match_faces(image_path, threshold, known_face_encodings, known_face_names)
    results.append((os.path.basename(image_path), matched_name, matched_index))
    if matched_name != "Unknown":
        if matched_name in aadhaar_details and matched_name not in recognized_faces and matched_name != "Nihar":
            recognized_faces.add(matched_name)
    display_results(results)

# Function to display matching results
def display_results(results):
    result_text = ""
    for filename, name, matched_index in results:
        result_text += f"Image: {filename} - Matched Name: {name}\n"
        if matched_index is not None:
            show_matching_images(filename, matched_index)
    result_label.config(text=result_text)

# Function to show the two matching images
def show_matching_images(filename, matched_index):
    # Load images
    matched_image_path = os.path.join(from_image_folder, known_face_names[matched_index] + '.jpg')
    to_image_path = os.path.join(to_image_folder, filename)

    # Debugging statements
    print(f"Loading matched image from: {matched_image_path}")
    print(f"Loading to image from: {to_image_path}")
    
    matched_image = cv2.imread(matched_image_path)
    to_image = cv2.imread(to_image_path)
    
    # Display images
    cv2.imshow('Matched Image', matched_image)
    cv2.imshow('To Image', to_image)
    cv2.waitKey(0)  # Wait for any key press
    cv2.destroyAllWindows()

# Function to update threshold and process selected image
def update_threshold():
    global stop_flag
    try:
        threshold = float(threshold_entry.get())
        if not stop_flag:
            image_path = filedialog.askopenfilename(title="Select Image to Match", filetypes=[("Image Files", "*.jpg;*.png")])
            if image_path and not stop_flag:
                process_selected_image(image_path, threshold)
    except ValueError:
        result_label.config(text="Please enter a valid threshold value (0.0 to 1.0).")
    except Exception as e:
        print(f"Error: {e}")

# Function to monitor key presses
def monitor_key_presses():
    global stop_flag
    while True:
        if keyboard.is_pressed('q'):
            stop_flag = True
            root.quit()
            break

# Initialize Tkinter
root = Tk()
root.title("Face Matching")
root.geometry("400x400")

threshold_label = Label(root, text="Set face matching threshold (0.0 to 1.0):")
threshold_label.pack()

threshold_entry = Entry(root)
threshold_entry.pack()
threshold_entry.insert(0, "0.6")  # Default threshold

threshold_button = Button(root, text="Update Threshold", command=update_threshold)
threshold_button.pack()

result_label = Label(root, text="")
result_label.pack()

# Load known faces
from_image_folder = 'from_image_folder'
to_image_folder = 'to_image_folder'
recognized_faces = set()  # Set to keep track of recognized faces

known_face_encodings, known_face_names, aadhaar_details = load_known_faces(from_image_folder)

# Start the key press monitoring thread
key_monitor_thread = threading.Thread(target=monitor_key_presses)
key_monitor_thread.daemon = True
key_monitor_thread.start()

# Function to check for the stop flag and close the window
def check_stop_flag():
    if stop_flag:
        root.destroy()
    else:
        root.after(100, check_stop_flag)

# Start checking for the stop flag
root.after(100, check_stop_flag)

# Start the Tkinter event loop
root.mainloop()

# Clean up
cv2.destroyAllWindows()


Loading matched image from: from_image_folder\ankursir_photo.jpg
Loading to image from: to_image_folder\ankursirnewphoto.jpg


: 